## Setup Imports

(not relevant to repo, can skip) 

Adding module path to run correctly in examples folder

In [2]:
import sys
import os
import numpy as np

# Add the path to the folder containing the module
module_path = os.path.abspath(os.path.join('../..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)

/Users/gracekim/Documents/School_Everything_and_LEARNING/Stanford/Githubs/loc-gsopt/src


In [3]:
from common.plotting import plot_gif,plot_img
import wandb
import math

api = wandb.Api()
run = api.run("loc_gsopt/teleport_select_100_data_downlink_10_5=CAPELLA=3000000/345h0ce0")
# run = api.run("loc_gsopt/teleport_select_100_data_downlink_7_5=CAPELLA=3000000/rnloat3a") #"<entity>/<project>/<run_id>")
# plot_img(run.summary.gs_list,f"gs_all.png")
# plot_img(run.summary.lat_long,f"images/gs_all.png")


# runs = api.runs("loc_gsopt/free_nelder_data_downlink_5_6=CAPELLA=500000000")
# for index, run in enumerate(runs):
#     plot_img(run.summary.gs_list,f"gs_all_{index}.png")

### Yaml

In [4]:
from omegaconf import OmegaConf
import yaml

# YAML configuration as a string
yaml_config = """
test_name: ""

wandb:
  entity: "loc_gsopt"

problem:
  type: "free"
  method: "nelder"
  objective: "data_downlink"
  gs_num: 10
  teleport_num: 10
  sat_num: 5
  data_rate: 1200000000

scenario:
  constellations: "CAPELLA"
  start_simplex: 3
  datarate: 1200000000.0
  teleport_sites: {}

constraints:
  dist_other_gs: 3000000

start_epoch:
  year: 2025
  month: 4
  day: 1
  hour: 17
  minute: 23
  second: 40.69

end_epoch:
  year: 2025
  month: 4
  day: 8
  hour: 17
  minute: 23
  second: 40.69

debug:
  verbose: True
  plot: True
  wandb: True
  txtUpdate: False
  randseed: 0

experiments:
  simplexExclude: True
  ccgs: 3
"""

# Load YAML into a Python dictionary
config_dict = yaml.safe_load(yaml_config)

# Convert the dictionary into an OmegaConf object
cfg = OmegaConf.create(config_dict)

# Access the OmegaConf object
print(OmegaConf.to_yaml(cfg))  # Print the configuration in YAML format

test_name: ''
wandb:
  entity: loc_gsopt
problem:
  type: free
  method: nelder
  objective: data_downlink
  gs_num: 10
  teleport_num: 10
  sat_num: 5
  data_rate: 1200000000
scenario:
  constellations: CAPELLA
  start_simplex: 3
  datarate: 1200000000.0
  teleport_sites: {}
constraints:
  dist_other_gs: 3000000
start_epoch:
  year: 2025
  month: 4
  day: 1
  hour: 17
  minute: 23
  second: 40.69
end_epoch:
  year: 2025
  month: 4
  day: 8
  hour: 17
  minute: 23
  second: 40.69
debug:
  verbose: true
  plot: true
  wandb: true
  txtUpdate: false
  randseed: 0
experiments:
  simplexExclude: true
  ccgs: 3



### CHecking Vedant

In [5]:
from common.utils import load_earth_data,mp_compute_contact_times,contactExclusion # compute_earth_interior_angle
from common.sat_gen import satellites_from_constellation
import brahe as bh
from common.station_gen import return_bdm_gs

# Make sure to load in earth inertial data every start time!
load_earth_data('data/iau2000A_finals_ab.txt',False)

satellites = satellites_from_constellation("CAPELLA", False)

epc_start = bh.Epoch(2025, 
                         4, 
                         1, 
                         17, 
                         23, 
                         40.69) 
    
epc_end = bh.Epoch(2025, 
                         4, 
                         8, 
                         17, 
                         23, 
                         40.69)  
    

gs_list = []
for new_gs in run.summary.lat_long:
    gs_list.append(return_bdm_gs(new_gs[0], new_gs[1]))

contacts, _ = mp_compute_contact_times(satellites, gs_list ,epc_start, epc_end, False)
_, contacts_exclusion_secs = contactExclusion(contacts,cfg)
# run.summary["output_data"] = gs_list_plot 
# run.summary["contact_num"] = len(contacts_exclusion_secs) 
# run.summary["seconds"] = np.sum(contacts_exclusion_secs)
# run.summary["data_downlink"] = np.sum(contacts_exclusion_secs)*cfg.scenario.datarate



Loading the latest Earth Orientation Data


In [6]:
gs_list

[Station(type='Feature', geometry=GeoJSONGeometry(type='Point', coordinates=[18.71868, -34.02762, 0.0]), properties=StationProperties(id=UUID('08af307e-e01f-4fd1-b9a7-31fece034a65'), station_name='', constraints=AccessConstraints(ascdsc=<AscendingDescending.either: 'either'>, look_direction=<LookDirection.either: 'either'>, look_angle_min=0.0, look_angle_max=90.0, elevation_min=10.0, elevation_max=90.0), downlink_rate_max=0.0)),
 Station(type='Feature', geometry=GeoJSONGeometry(type='Point', coordinates=[-157.8664848, 21.31494, 0.0]), properties=StationProperties(id=UUID('04812e7a-5c62-4255-9335-14997c80d493'), station_name='', constraints=AccessConstraints(ascdsc=<AscendingDescending.either: 'either'>, look_direction=<LookDirection.either: 'either'>, look_angle_min=0.0, look_angle_max=90.0, elevation_min=10.0, elevation_max=90.0), downlink_rate_max=0.0)),
 Station(type='Feature', geometry=GeoJSONGeometry(type='Point', coordinates=[141.2583617, 43.1732192, 0.0]), properties=StationProp

In [7]:
np.sum(contacts_exclusion_secs)*cfg.scenario.datarate

792912192673200.0

In [8]:
run.summary["output_data"]

792912192673200

In [6]:
run.summary.lat_long

[[-111.951599, 40.783137]]